In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from sklearn.linear_model import LinearRegression

# Point prediction  
1. Explore data (visualize, graphs etc.)  
2. Split dataset into train-validate-test 
3. Baseline model, mean points 
4. Train one or more model and evaulate performance on validation data (rmse ?)
5. Maybe do some feature engineering  

In [ ]:
df = pd.read_csv('../input/fantasy-premier-league-20182019/FPL_2018_19_Wk0.csv')
df1 = pd.read_csv('../input/fantasy-premier-league-20182019/FPL_2018_19_Wk1.csv')
df2 = pd.read_csv('../input/fantasy-premier-league-20182019/FPL_2018_19_Wk2.csv')
df3 = pd.read_csv('../input/fantasy-premier-league-20182019/FPL_2018_19_Wk3.csv')
df4 = pd.read_csv('../input/fantasy-premier-league-20182019/FPL_2018_19_Wk4.csv')
df5 = pd.read_csv('../input/fantasy-premier-league-20182019/FPL_2018_19_Wk5.csv')
df6 = pd.read_csv('../input/fantasy-premier-league-20182019/FPL_2018_19_Wk6.csv')
df7 = pd.read_csv('../input/fantasy-premier-league-20182019/FPL_2018_19_Wk7.csv')
df8 = pd.read_csv('../input/fantasy-premier-league-20182019/FPL_2018_19_Wk8.csv')
df9 = pd.read_csv('../input/fantasy-premier-league-20182019/FPL_2018_19_Wk9.csv')
df10 = pd.read_csv('../input/fantasy-premier-league-20182019/FPL_2018_19_Wk10.csv')
df11 = pd.read_csv('../input/fantasy-premier-league-20182019/FPL_2018_19_Wk11.csv')


In [ ]:
df

Week 0 looks like it is either week 12 or accumulated numbers from the previous season. Lets look at week 1 instead.

In [ ]:
df1

In [ ]:
df2

Week 1 has 525 rows while week 2 has 529 rows.  
We need to find out what the differences are before we continue.

In [ ]:
wk1_dup = df1.Name.value_counts()
wk2_dup = df2.Name.value_counts()

We want to look for either duplicates or missing data in order to identify the differences between gameweek 1 and gameweek 2.

In [ ]:
for i in wk2_dup.index:
    if i not in wk1_dup.index or wk2_dup[i] != wk1_dup[i]:
        print(i)

In [ ]:
for i in wk1_dup.index:
    if i not in wk2_dup.index or wk2_dup[i] != wk1_dup[i]:
        print(i)

In [ ]:
df1.loc[df1.Name == 'Mendy']

In [ ]:
df2.loc[df2.Name == 'Mendy']

In gameweek 2 a Leicester player named Mendy appears, although he is not registered in gameweek 1.  
Looking at the column Minutes, we see that he has played 180 minutes which is the equivalent of two whole matches.  
Our dataset has missing data in some degree.

In [ ]:
df1.loc[df1.Name == 'Dowell']

In [ ]:
df2.loc[df2.Name == 'Dowell']

In [ ]:
df1.Minutes.value_counts()

In [ ]:
df2.Minutes.value_counts()

In [ ]:
uni_names = []
for i in range(1,12):
    df = pd.read_csv('../input/fantasy-premier-league-20182019/FPL_2018_19_Wk' +str(i) +'.csv')
    uni_names.append(df.Name.unique())
    print(len(df))
    

Almost every gameweek has a different number of rows.  
A simple solution would be to only keep players listed in every gameweek and drop the rest.  
But first we need to look at every gameweek in order to get a complete list of players.  


In [ ]:
a = np.array(uni_names,dtype=object)
name = np.unique(np.concatenate(a))
name

Lets doublecheck that 536 unique player names is a sensible number

In [ ]:
for i in uni_names:
    print(len(i))

The final step is to filter out all players not occuring in every gameweek.

In [ ]:
bad_name = []
for i in name:
    if i not in df1.Name.values or \
    i not in df2.Name.values or \
    i not in df3.Name.values or \
    i not in df4.Name.values or \
    i not in df5.Name.values or \
    i not in df6.Name.values or \
    i not in df7.Name.values or \
    i not in df8.Name.values or \
    i not in df9.Name.values or \
    i not in df10.Name.values or \
    i not in df11.Name.values:
        print(i)
        bad_name.append(i)

Jordan Ayew is listed as both "Ayew" and "Jordan Ayew" in our dataset. Maybe we can rename all occurences to either.

In [ ]:
dfs = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11]
for i in dfs:
    if (i.Name == "Ayew").any():
        print("Ayew")
    elif (i.Name == "Jordan Ayew").any():
        print("Jordan Ayew")

In total we have 11 occurences where 10 are "Ayew". In gameweek 1 he is listed as "Jordan Ayew", lets rename it to "Ayew" and remove him from the list of names to be dropped.  

In [ ]:
df1.loc[df1.Name == 'Jordan Ayew']

In [ ]:
df1.at[235,'Name'] = "Ayew"
bad_name.remove('Ayew')
print(bad_name)
df1.at[235,'Name']

In [ ]:
dfs = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11]
for i in dfs:
    i.set_index('Name', inplace = True)

for i in bad_name:
    for dataframe in dfs:
        if i in dataframe.index:
            dataframe.drop([i], inplace = True)

Now all of our gameweeks should be filtered and of same size. Lets check if they are.

In [ ]:
for i in dfs:
    print(len(i))

Not quite, something is missing from gameweek 1.

In [ ]:
for i in name:
    if i not in bad_name and df1.index.value_counts()[i] != df2.index.value_counts()[i]:
        print(i)

We forgot to account for the case we observed in the beginning (missing data & duplicate names), luckily it seems like there are no other such cases.  
Since the other "Mendy" playing for Leicester is only missing from gameweek 1 we manually add him to our gameweek 1 dataset.

In [ ]:
temp = df2.copy()
temp.reset_index(inplace = True)
mendy = temp.loc[327]
inp = pd.DataFrame(mendy).transpose()
inp.at[327,'Goals_conceded'] = 0
inp.at[327,'Minutes'] = 90
inp.at[327,'Points'] = 0
inp.set_index('Name',inplace = True)
df1 = df1.append(inp)

In [ ]:
corr = df11.corr()
corr.style.background_gradient(cmap='coolwarm')

Now, we should be able to start predicting points.

In [ ]:
def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)

In [ ]:
def get_dummies(df): 
    df = pd.get_dummies(df,columns = ['Position'],prefix ='',prefix_sep='')
    df = pd.get_dummies(df,columns = ['Team'],prefix ='',prefix_sep='')
    return df

In [ ]:
def get_valset(df):
    val = []
    df_x = get_dummies(df)
    df_y = df_x.Points.values
    df_x.drop(columns = ['Points'],inplace = True)
    val.append(df_x)
    val.append(df_y)
    return val

In [ ]:
def get_one_week_model(x1):
    x_train = x1
    y_train = x_train['Points'].values
    x_train.drop(columns = ['Points'],inplace = True)
    lr = LinearRegression()
    lr.fit(x_train,y_train)
    return lr

In [ ]:
def get_model(x1,x2):
    x_train = x1.append(x2)
    y_train = x_train['Points'].values
    x_train.drop(columns = ['Points'],inplace = True)
    lr = LinearRegression()
    lr.fit(x_train,y_train)
    return lr

In [ ]:
def get_performance(model,val):
    return r_mse(np.clip(model.predict(val[0]),0,None),val[1])

In [ ]:
get_performance(get_model(get_dummies(df1),get_dummies(df2)),get_valset(df3))

In [ ]:
get_performance(get_model(get_dummies(df4),get_dummies(df5)),get_valset(df6))

In [ ]:
get_performance(get_model(get_dummies(df7),get_dummies(df8)),get_valset(df9))

In [ ]:
get_performance(get_one_week_model(get_dummies(df10)),get_valset(df11))

In [ ]:
x_train = df1.append(df2).append(df3).append(df4).append(df5).append(df6).append(df7).append(df8)
y_train = x_train['Points'].values

In [ ]:
x_train = pd.get_dummies(x_train,columns = ['Position'],prefix ='',prefix_sep='')
x_train = pd.get_dummies(x_train,columns = ['Team'],prefix ='',prefix_sep='')
x_train

In [ ]:
lr = LinearRegression()
x_train.drop(columns = ['Points'],inplace = True)
lr.fit(x_train,y_train)

In [ ]:
x_val = df9.append(df10).append(df11)
y_val = x_val.Points.values

In [ ]:
x_val = pd.get_dummies(x_val,columns = ['Position'],prefix ='',prefix_sep='')
x_val = pd.get_dummies(x_val,columns = ['Team'],prefix ='',prefix_sep='')
x_val.drop(columns = ['Points'],inplace = True)

In [ ]:
r_mse(np.clip(lr.predict(x_val),0,None),y_val)

In [ ]:
x_val['Prediction'] = np.clip(lr.predict(x_val),0,100)
x_val['Actual'] = y_val
x_val

In [ ]:
x_val[x_val.index == 'Aguero'][['Assists','Goals_scored','Minutes','Prediction','Actual']]

In [ ]:
cdf = pd.DataFrame(lr.coef_,x_train.columns, columns=['Coefficients'])
cdf.plot(kind ="bar" ,figsize = (20,10))

Since we are dealing with many continuous variables we should have normalized them prior to training.  
An "Oppnent_team" feature would probably be very helpful.
